<a href="https://colab.research.google.com/github/karsarobert/ChatGpt_2024/blob/main/09/ChatGPT2024_09_FELADATOKKAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chat GPT és más nagy nyelvi modellek alkalmazása
##PTE Gépi tanulás
###9. gyakorlat: Langchain 1. rész
2024. április 8.

A jegyzetfüzethez kell OpenAI és Huggingface API kulcs!

In [ ]:
!pip install langchain openai tiktoken faiss-cpu sentence-transformers


#Modellek és promptok
A LangChain több mint 50 integrációt kínál külső gyártók és platformok felé, beleértve az OpenAI-t, az Azure OpenAI-t, a Databricks-et és a MosaicML-t, valamint az integrációt a Hugging Face Hub-bal és a nyílt forráskódú LLM-ek világával.

In [ ]:
from google.colab import userdata


In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(api_key = userdata.get('openai_api_key'))
print(llm('tell me a joke'))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




Why did the tomato turn red?

Because it saw the salad dressing!


##Prompt sablonok.
A prompt sablon egy olyan elem, ami meghatározza, hogyan lehet létrehozni egy utasítást (promptot) egy nyelvi modell számára. Tartalmazhat változókat, helyőrzőket, elő- és utótagokat, illetve más elemeket, amelyek az adatoknak és a feladatnak megfelelően testreszabhatók.

In [ ]:
from langchain import PromptTemplate
template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])
print(prompt.format(sentence = "the cat is on the table", language = "hungarian"))

Sentence: the cat is on the table
Translation in hungarian:


Általánosságban elmondható, hogy a prompt sablonok jellemzően függetlenek attól, hogy végül melyik LLM-et használjuk, és mind a befejezési (completion), mind a chat modellekhez alkalmazhatóak.

Befejezési modell (completion model): Ez egy olyan LLM-típus, amely szöveges inputot kap, és szöveges outputot generál (ezt nevezzük befejezésnek). A befejezési modell megpróbálja koherens és a témához illő módon folytatni a promptot, annak megfelelően, hogy milyen feladatra és milyen adatokra képezték ki. Például egy befejezési modell generálhat összefoglalásokat, fordításokat, történeteket, kódot, dalszöveget és még sok mást a prompttól függően.

Chat modell: A befejezési modell egy speciális típusa, amelyet beszélgetős válaszok generálására terveztek. A chat modell bemenetként egy üzenetlistát kap, ahol minden üzenet rendelkezik egy szereppel (rendszer, felhasználó vagy asszisztens) és tartalommal. A chat modell megpróbál új üzenetet generálni az asszisztens szerepéhez, a korábbi üzenetek és a rendszer utasítása alapján.

A fő különbség a befejezési és a chat modellek között az, hogy a befejezési modellek egyetlen szöveges bemenetet várnak promptként, míg a chat modellek bemenetként üzenetek listáját.

Példaválasztó (Example selector). A LangChainben a példa kiválasztó egy olyan komponens, amellyel kiválaszthatod, mely példákat illeszd be a nyelvi modell promptjába. A prompt egy olyan szöveges bemenet, amely a nyelvi modellt a kívánt kimenet előállítására irányítja. A példák olyan bemeneti/kimeneti párok, amelyek bemutatják a feladatot és a kimenet formátumát.

In [ ]:
import yfinance as yf
import pandas as pd
# Define the ticker symbols
tickers = ['AAPL']
# Define the start and end dates
start_date = '2020-01-01'
end_date = '2022-12-31'
# Create an empty DataFrame to store the data
data = pd.DataFrame()
# Download the data
for ticker in tickers:
 df = yf.download(ticker, start=start_date, end=end_date, interval='1mo')
 df['Ticker'] = ticker # Add a column with the ticker symbol
 data = pd.concat([data, df])
# Reset the index
data.reset_index(inplace=True)
# Save the data to a CSV file
data.to_csv('stock_data.csv', index=False)

[*********************100%%**********************]  1 of 1 completed


##Dokumentum betöltők (Document loaders).
Ezek felelősek a dokumentumok betöltéséért különböző forrásokból, mint például CSV, fájlkönyvtárak (File Directory), HTML, JSON, Markdown és PDF. A dokumentum betöltők egy .load metódust biztosítanak, amellyel adatokat tölthetsz be dokumentumokként egy konfigurált forrásból. A kimenet egy Document objektum, ami egy szövegrészletet és a hozzá tartozó metaadatokat tartalmazza.

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='stock_data.csv')
data = loader.load()
print(data)

[Document(page_content='Date: 2020-01-01\nOpen: 74.05999755859375\nHigh: 81.9625015258789\nLow: 73.1875\nClose: 77.37750244140625\nAdj Close: 75.2875747680664\nVolume: 2934370400\nTicker: AAPL', metadata={'source': 'stock_data.csv', 'row': 0}), Document(page_content='Date: 2020-02-01\nOpen: 76.07499694824219\nHigh: 81.80500030517578\nLow: 64.09249877929688\nClose: 68.33999633789062\nAdj Close: 66.49418640136719\nVolume: 3019279200\nTicker: AAPL', metadata={'source': 'stock_data.csv', 'row': 1}), Document(page_content='Date: 2020-03-01\nOpen: 70.56999969482422\nHigh: 76.0\nLow: 53.15250015258789\nClose: 63.5724983215332\nAdj Close: 62.00223922729492\nVolume: 6280072400\nTicker: AAPL', metadata={'source': 'stock_data.csv', 'row': 2}), Document(page_content='Date: 2020-04-01\nOpen: 61.625\nHigh: 73.63249969482422\nLow: 59.224998474121094\nClose: 73.44999694824219\nAdj Close: 71.63575744628906\nVolume: 3265299200\nTicker: AAPL', metadata={'source': 'stock_data.csv', 'row': 3}), Document(pa

Dokumentum átalakítók (Document transformers).
A dokumentumok importálása után gyakori, hogy módosítani kell őket, hogy jobban megfeleljenek az igényeidnek. Erre egy alapvető példa, amikor egy hosszú dokumentumot kisebb részekre bontasz, amelyek illeszkednek a modelled kontextusablakába (a modell által egyszerre feldolgozható szövegmennyiségbe). A LangChain-en belül különböző, előre elkészített dokumentum átalakítók állnak rendelkezésre, amelyeket szövegfelosztóknak (Text splitters) neveznek. A szövegfelosztók célja, hogy megkönnyítsék a dokumentumok szemantikailag kapcsolódó részekre bontását, így nem veszítünk kontextust vagy releváns információt.

A szövegfelosztókkal eldöntheted, hogyan oszd fel a szöveget (pl. karakter, címsor, token stb. alapján), és hogyan mérd meg a szövegrészlet hosszát (pl. a karakterek száma alapján).

In [ ]:
!gdown 183bdW5i6KCv5fq_nZGx3nOIi6N70gZ0x

Downloading...
From: https://drive.google.com/uc?id=183bdW5i6KCv5fq_nZGx3nOIi6N70gZ0x
To: /content/piszkosfred_reszlet.txt
100% 5.22k/5.22k [00:00<00:00, 8.99MB/s]


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("piszkosfred_reszlet.txt")
text = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 200,
  chunk_overlap = 50,
  length_function = len
)

texts = text_splitter.split_documents(text)
print(texts[0])
print(texts[1])
print(texts[2])

page_content='\ufeff\nRejtő Jenő\n\nPiszkos Fred, a kapitány' metadata={'source': 'piszkosfred_reszlet.txt'}
page_content='ELSŐ FEJEZET\n1\n- Uram! A késemért jöttem!\n- Hol hagyta?\n- Valami matrózban.\n- Milyen kés volt?\n- Acél. Keskeny penge, kissé hajlott. Nem látta?' metadata={'source': 'piszkosfred_reszlet.txt'}
page_content='- Acél. Keskeny penge, kissé hajlott. Nem látta?\n- Várjunk... Csak lassan, kérem... Milyen volt a nyele?\n- Kagyló.\n- Hány részből?\n- Egy darabból készült.\n- Akkor nincs baj. Megvan a kés!\n- Hol?' metadata={'source': 'piszkosfred_reszlet.txt'}


In [ ]:
with open('piszkosfred_reszlet.txt') as f:
  text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 200,
  chunk_overlap = 50,
  length_function = len
)
texts = text_splitter.create_documents([text])
print(texts[0])
print(texts[1])
print(texts[2])

page_content='\ufeff\nRejtő Jenő\n\nPiszkos Fred, a kapitány'
page_content='ELSŐ FEJEZET\n1\n- Uram! A késemért jöttem!\n- Hol hagyta?\n- Valami matrózban.\n- Milyen kés volt?\n- Acél. Keskeny penge, kissé hajlott. Nem látta?'
page_content='- Acél. Keskeny penge, kissé hajlott. Nem látta?\n- Várjunk... Csak lassan, kérem... Milyen volt a nyele?\n- Kagyló.\n- Hány részből?\n- Egy darabból készült.\n- Akkor nincs baj. Megvan a kés!\n- Hol?'


##Szövegbeágyazó modellek (Text embedding models).
A beágyazás a szavak, részek vagy karakterek folyamatos vektortérben történő ábrázolásának módját jelenti.

A beágyazások kulcsfontosságú lépést jelentenek a nem-parametrikus tudás beépítésében az LLM-ekbe. Valójában, miután megfelelően tárolták őket egy VectorDB-ben, válnak azzá a nem-parametrikus tudássá, amellyel szemben mérni tudjuk a felhasználó lekérdezésének távolságát.

A beágyazás használatának megkezdéséhez szükség lesz egy beágyazási modellre:

Ezután a LangChain két fő modullal rendelkező Embedding osztályt kínál, amelyek a nem-parametrikus tudás (több bemeneti szöveg), illetve a felhasználói lekérdezés (egyetlen bemeneti szöveg) beágyazását célozzák.

Vegyünk például egy beágyazást az OpenAI text-embedding-ada-002 beágyazási modelljével (az OpenAI beágyazási modelljeivel kapcsolatos további részletekért az hivatalos dokumentációt a következő címen érheted el: https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model ='text-embedding-ada-002' ,api_key = userdata.get('openai_api_key'))
embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)
print("Embed documents:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
print("Embed query:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Embed documents:
Number of vector: 5; Dimension of each vector: 1536
Embed query:
Dimension of the vector: 1536
Sample of the first 5 elements of the vector: [0.0053848074247278025, -0.0005522561790177142, 0.038960665101309515, -0.0029398672940039064, -0.008987877434176594]


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_modelHF = HuggingFaceEmbeddings()

embeddingsHF = embeddings_modelHF.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)
print("Embed documents:")
print(f"Number of vector: {len(embeddingsHF)}; Dimension of each vector: {len(embeddingsHF[0])}")
embedded_query = embeddings_modelHF.embed_query("What was the name mentioned in the conversation?")
print("Embed query:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embed documents:
Number of vector: 5; Dimension of each vector: 768
Embed query:
Dimension of the vector: 768
Sample of the first 5 elements of the vector: [0.09514584392309189, 9.877218690235168e-05, -0.016573388129472733, 0.044848017394542694, 0.04323701933026314]


Miután beágyaztuk mind a dokumentumokat, mind a lekérdezést, a következő lépés a két elem közötti hasonlóság kiszámítása, és a legmegfelelőbb információ kinyerése a dokumentumok beágyazásából.

Vektortárolók (Vector stores). A vektortároló (vagy VectorDB) egy olyan adattbázis-típus, amely strukturálatlan adatokat, például szöveget, képeket, hangot vagy videót tud tárolni és azokon keresni beágyazások (embedding-ek) segítségével. A beágyazások használatával a vektortárolók gyors és pontos hasonlósági keresést tudnak végrehajtani, ami azt jelenti, hogy megtalálják a legrelevánsabb adatokat egy adott lekérdezéshez.

In [ ]:
!gdown 1tICd5l-gv1PhZgvCkLD4ydqq-jIaHUv-

Downloading...
From: https://drive.google.com/uc?id=1tICd5l-gv1PhZgvCkLD4ydqq-jIaHUv-
To: /content/dialog.txt
100% 114/114 [00:00<00:00, 369kB/s]


##OpenAi beágyazásokkal

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('dialog.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings(api_key = userdata.get('openai_api_key')))

In [ ]:
query = "What is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

I want to report an accident


##Huggingface alapmodell beágyazásokkal

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

embeddings_modelHF = HuggingFaceEmbeddings()

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('dialog.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
dbHF = FAISS.from_documents(documents, embeddings_modelHF)

In [ ]:
query = "What is the reason for calling?"
docs = dbHF.similarity_search(query)
print(docs[0].page_content)
print(docs[1].page_content)

Good morning!
Oh, hello!
I want to report an accident


##Huggingface e5-large modell beágyazásokkal

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

embeddings_modelHF = HuggingFaceEmbeddings(model_name = 'intfloat/e5-large')

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('dialog.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
dbHF2 = FAISS.from_documents(documents, embeddings_modelHF)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
query = "What is the reason for calling?"
docs = dbHF2.similarity_search(query)
print(docs[0].page_content)
print(docs[1].page_content)

Sorry to hear that. May I ask your name?
I want to report an accident


##Lekérdezők (Retrievers).
A LangChainben a retriever olyan komponens, amely képes egy strukturálatlan lekérdezésre releváns dokumentumokat visszaadni, például egy természetes nyelvi kérdésre vagy kulcsszóra. A retrievernek nem kell magának tárolnia a dokumentumokat, csak egy forrásból kell visszakeresnie őket. A retriever különböző módszereket használhat a releváns dokumentumok megtalálására, például kulcsszóegyeztetést, szemantikai keresést vagy rangsorolási algoritmusokat.

A retriever és a vektortároló (vector store) közötti különbség az, hogy a retriever általánosabb és rugalmasabb. A retriever bármilyen módszerrel megtalálhatja a releváns dokumentumokat, míg a vektortároló beágyazásokra és hasonlósági metrikákra támaszkodik. A retriever különböző dokumentumforrásokat is használhat, például weboldalakat, adatbázisokat vagy fájlokat, míg a vektortárolónak magának kell tárolnia az adatokat.

Azonban a vektortároló használható retriever alapjaként is, ha az adatokat beágyazza és indexeli egy vektortároló. Ebben az esetben a retriever a vektortárolót használhatja hasonlósági keresés elvégzésére a beágyazott adatokon, és visszaadhatja a legrelevánsabb dokumentumokat.  Ez az egyik fő típusa a LangChainben használt retrievereknek, és vektoros tárolón alapuló retrievernek (vector store retriever) hívjuk.

Például vegyük a korábban inicializált FAISS vektortárolót, és "szereljünk fel" rá egy retrievert.

###OpenAi beágyazásokat használva

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=OpenAI(api_key = userdata.get('openai_api_key')), chain_type="stuff", retriever=retriever)
query = "What was the reason of the call?"
qa.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The reason for the call was to report an accident.'

###Ingyenes Huggingface beágyazásokat használva

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
retriever = dbHF2.as_retriever()
qa = RetrievalQA.from_chain_type(llm=OpenAI(api_key = userdata.get('openai_api_key')), chain_type="stuff", retriever=retriever)
query = "What was the reason of the call?"
qa.run(query)

' The reason for the call was to report an accident. '

In [ ]:
retriever.get_relevant_documents("What was the reason of the call?")

[Document(page_content='Sorry to hear that. May I ask your name?', metadata={'source': 'dialog.txt'}),
 Document(page_content='I want to report an accident', metadata={'source': 'dialog.txt'}),
 Document(page_content='Good morning!\nOh, hello!', metadata={'source': 'dialog.txt'}),
 Document(page_content='Sure, Mario Rossi.', metadata={'source': 'dialog.txt'})]

In [ ]:
from langchain import HuggingFaceHub
repo_id = "mistralai/Mistral-7B-Instruct-v0.2" #"tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 1000}, huggingfacehub_api_token=userdata.get('HF_TOKEN')
)
print(llm("what was the first disney movie?"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


what was the first disney movie?

The first Disney movie was actually a series of short cartoons called "Steamboat Willie." It was released on November 18, 1928, and it featured Mickey Mouse for the very first time. This was a significant breakthrough for animation because Steamboat Willie was the first cartoon to synchronize sound with the movement of the characters' lips, a technique known as synchronized sound or "talking" cartoons. This innovation revolution


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
retriever = dbHF2.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
query = "What was the reason of the call?"
answer = qa.run(query)

In [ ]:
answer

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nSorry to hear that. May I ask your name?\n\nI want to report an accident\n\nGood morning!\nOh, hello!\n\nSure, Mario Rossi.\n\nQuestion: What was the reason of the call?\nHelpful Answer: The caller identified himself as Mario Rossi and stated that he wanted to report an accident."

##Láncok (Chains)
A láncok előre meghatározott műveleti és LLM-hívási sorozatok, amelyek megkönnyítik az összetett alkalmazások létrehozását. Ehhez szükség lehet az LLM-ek egymással vagy más komponensekkel való kombinálására. A LangChain négy fő lánctípust kínál, hogy könnyen elkezdhesd a munkát:

LLMChain. Ez a leggyakoribb lánctípus. Egy PromptTemplate-et (prompt sablon), egy LLM-et és egy opcionális output parser-t (kimeneti elemző) tartalmaz.

Definíció
A kimeneti elemző (output parser) egy olyan komponens, amely segít a nyelvi modell válaszait strukturálni. Ez egy olyan osztály, amely két fő metódust valósít meg: get_format_instructions és parse. A get_format_instructions metódus egy olyan karakterláncot ad vissza, amely utasításokat tartalmaz arra vonatkozóan, hogyan kell formázni a nyelvi modell kimenetét. A parse metódus egy karakterláncot vesz be (amely feltételezhetően egy nyelvi modell válasza), és valamilyen struktúrává elemzi, például szótárrá, listává vagy egyéni objektummá.

Ez a lánc több bemeneti változót fogad, a PromptTemplate segítségével prompt-tá formázza őket, átadja a modellnek, majd az OutputParser-t használja (ha meg van adva), hogy a LLM kimenetét végleges formátumba elemezze. Például vegyük elő azt a prompt sablont, amit az előző részben készítettünk.



In [ ]:
from langchain import PromptTemplate
template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

###OpenAi


In [ ]:
from langchain import OpenAI, LLMChain
llm = OpenAI(api_key = userdata.get('openai_api_key'), temperature=0)
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.predict(sentence="the cat is on the table", language="magyar")

' a macska az asztalon van'

##Huggingface Mixtral modell

In [ ]:
from langchain import OpenAI, LLMChain
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 1000}, huggingfacehub_api_token=userdata.get('HF_TOKEN')
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
answer= llm_chain.predict(sentence="the cat is on the table", language="magyar")

In [ ]:
answer

'Sentence: the cat is on the table\nTranslation in magyar: a macska az asztalon van\n\nWhy is the cat on the table?\nWhy is it always the cat, and not the dog?\nWhy a table? Why not a chair or an armchair?\nDoes the cat have business on the table?\nHow did the cat get on the table?\n\nThese are questions that arose in my mind when I saw the sentence on an exercise.\n\nThe cat is on the table.\n\nI can imagine'

##SequentialChain (Szekvenciális lánc).
Ez a lánctípus lehetővé teszi, hogy több láncot futtass sorrendben. Megadhatod a láncok sorrendjét és azt is, hogyan adják át a kimeneteiket a következő láncnak. A szekvenciális lánc legegyszerűbb modulja alapértelmezetten az egyik lánc kimenetét használja a következő lánc bemeneteként. Azonban használhatsz egy összetettebb modult is, hogy rugalmasabban állíthasd be a láncok közötti bemeneteket és kimeneteket.

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(api_key = userdata.get('openai_api_key'), temperature=.7)
template = """You are a comedian. Generate a joke on the following {topic}
Joke:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template, output_key='out1')
joke_chain = LLMChain(llm=llm, prompt=prompt_template)
template = """You are translator. Given a text {out1}, translate it to hungarian
Translation:"""
prompt_template = PromptTemplate(input_variables=["out1"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("Cats and Dogs")



> Entering new SimpleSequentialChain chain...
 Why did the cat and dog decide to open a bakery together?

Because they wanted to make paw-some treats!
 Miért döntöttek úgy a macska és kutya, hogy együtt nyitnak egy pékséget?

Mert szerettek volna csodás finomságokat készíteni!

> Finished chain.


In [ ]:
translated_joke

' Miért mentek együtt a macska és a kutya terápiára?\nMert belefáradtak a saját farkuk üldözésébe és szükségük volt valakire, aki segít nekik megtalálni belső békéjüket!'

#Mixtral modell

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 1000}, huggingfacehub_api_token=userdata.get('HF_TOKEN')
)


template = """You are a comedian. Generate a joke on the following {topic}
Joke:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)
template = """You are translator. Given a text {input}, translate it to hungarian, no need to explanation
Translation:"""
prompt_template = PromptTemplate(input_variables=["input"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("Cats and Dogs")



> Entering new SimpleSequentialChain chain...
You are a comedian. Generate a joke on the following Cats and Dogs
Joke:

There once was a cat who thought he was a dog.
He would chase his tail, bark at the mailman, and even try to fetch the newspaper.
One day, his owner took him to the vet and said, "Doc, I think there's something wrong with my cat. He thinks he's a dog."
The vet looked at the cat and said, "Well, he may think he's a dog, but I can assure you he'
You are translator. Given a text You are a comedian. Generate a joke on the following Cats and Dogs
Joke:

There once was a cat who thought he was a dog.
He would chase his tail, bark at the mailman, and even try to fetch the newspaper.
One day, his owner took him to the vet and said, "Doc, I think there's something wrong with my cat. He thinks he's a dog."
The vet looked at the cat and said, "Well, he may think he's a dog, but I can assure you he', translate it to hungarian, no need to explanation
Translation:
Volt egyszer eg

In [ ]:
translated_joke

'You are translator. Given a text You are a comedian. Generate a joke on the following Cats and Dogs\nJoke:\n\nThere once was a cat who thought he was a dog.\nHe would chase his tail, bark at the mailman, and even try to fetch the newspaper.\nOne day, his owner took him to the vet and said, "Doc, I think there\'s something wrong with my cat. He thinks he\'s a dog."\nThe vet looked at the cat and said, "Well, he may think he\'s a dog, but I can assure you he\', translate it to hungarian, no need to explanation\nTranslation:\nVolt egyszer egy macska, aki úgy gondolta magáról, hogy kutya.\nŐ a farkát kergette, a postást ugatta ki, és még újságot is hozni próbált.\nEgy napon a tulajdonosa elvitte a vételre és azt mondta: "Doktor, a macskámnak valami ninc'

#Feladat
Az alábbi szövegfájlt töltsed le és készíts hozzá egy vektor adatbázist, majd tedd fel az alábbi kérdéseket:
* "Ki volt Nemecsek?"
* "Mi a gittegylet szerepe?"
* "Ki volt a vörösingesek vezetője?"

Nézzd meg a retriver által adott válaszokat is!


In [ ]:
!gdown 1op1icuNvSOmXzfrfFPfGGsi-BX4T6rNI

Downloading...
From: https://drive.google.com/uc?id=1op1icuNvSOmXzfrfFPfGGsi-BX4T6rNI
To: /content/PUF.txt
100% 295k/295k [00:00<00:00, 71.4MB/s]


#Feldadat
Készíts langchain segítségével egy láncot amely egy adott témával {topic} kapcsolatosan vissza adj a téma 4 legfontosabb területét,
majd a lánc második eleme ezt pontonként kifejti részletesen.